<a href="https://colab.research.google.com/github/WKoishi/deep_learning_colab/blob/main/mobilenet_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!/opt/bin/nvidia-smi

Tue Jun 22 06:18:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    28W /  70W |   2778MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
os.chdir('/content/drive/MyDrive')

!unzip dataset_face_0622.zip

In [ ]:
import os, datetime
os.chdir('/content/drive/MyDrive')
%load_ext tensorboard

import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras import callbacks
import os
import numpy as np

logdir = os.path.join("my_train_log", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

callbacks_list = [   
    # callbacks.ModelCheckpoint(   
    #     filepath='my_model.h5',   
    #     monitor='val_loss',   
    #     save_best_only=True, 
    # ), 
    callbacks.TensorBoard(
        log_dir = logdir,
    )
]

# 设置GPU显存
config = tf.compat.v1.ConfigProto(allow_soft_placement=True)
#config.gpu_options.per_process_gpu_memory_fraction = 0.8
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

img_shape = (128, 128)
train_dir = 'dataset_face_shuffle/train'
validation_dir = 'dataset_face_shuffle/validation'

#将训练数据进行增强
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,  #图像随机旋转的角度范围
    #width_shift_range=0.2,  #图像在水平或垂直方向上平移的范围（相对于高度或宽度的比例）
    #height_shift_range=0.2,
    zoom_range=0.1,  #随机缩放的范围
    horizontal_flip=True,  #随机将一半图像水平翻转
    vertical_flip=True,     #随机垂直翻转
    fill_mode='nearest',  #填充新创建的像素
)

#将验证数据进行轻度的增强
validation_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    #rotation_range=20,
    vertical_flip=True,
    horizontal_flip=True
)


#使用Python生成器
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = img_shape, 
    batch_size = 32,  #每个批量中包含32个样本
    class_mode ='categorical' 
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size = img_shape, 
    batch_size = 16,
    class_mode = 'categorical'
)


base_model = MobileNetV2(input_shape=(128,128,3), include_top=False,
                    weights='imagenet')

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)

predictions = layers.Dense(2, name='Logits')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# for i, layer in enumerate(model.layers):
#    print(i, layer.trainable, layer.name)

#print(model.summary())


model.compile(optimizer = optimizers.Adam(learning_rate=0.0005),
                loss = losses.CategoricalCrossentropy(from_logits=True),
                metrics=['acc'])

history=model.fit(train_generator,
          steps_per_epoch=92,
          epochs=35,
          validation_data=validation_generator,
          validation_steps=46,
          callbacks=callbacks_list)

#get_tf_model_proto(model)  #保存成pb文件
model.save('FR_classify_1.h5')






The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Found 2950 images belonging to 2 classes.
Found 736 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 1/35
92/92 [==============================] - 21s 194ms/step - loss: 0.7259 - acc: 0.6532 - val_loss: 0.8671 - val_acc: 0.6399
Epoch 2/35
92/92 [==============================] - 16s 176ms/step - loss: 0.4686 - acc: 0.7779 - val_loss: 0.6281 - val_acc: 0.7812
Epoch 3/35
92/92 [==============================] - 16s 176ms/step - loss: 0.4072 - acc: 0.8194 - val_loss: 0.6231 - val_acc: 0.7785
Epoch 4/35
92/92 [==============================] - 16s 173ms/step - loss: 0.3804 - acc: 0.8324 - val_loss: 0.6641 - val_acc: 0.7962
Epoch 5/35
92/92 [==============================] - 16s 174ms/step - loss: 0.3609 - acc: 0.8458 - val_loss: 0.8635 - val_acc: 0.7717
Epoch 6/35
92/92 [==============================] - 16s 176ms/step - loss: 0.3420 - acc: 0.8520 - val_loss: 0.5573 - val_acc: 0.8573
Epoch 7/35
92/92 [==============================] - 16s 174ms/step - loss: 0.3207 - acc: 0.8667 - val_loss: 0.8767 - val_acc: 0.7704
Epoch 8/35
92/92 [==============================] - 16s 174ms/step - 

In [ ]:
import os
os.chdir('/content/drive/MyDrive')
#%reload_ext tensorboard
%tensorboard --logdir my_train_log